## Exploring the Dataset

The goal of this project is to create a spam filter. We will utilize the computer and how it:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

The dataset we will be using was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details [on this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

In [1]:
import pandas as pd
sms_spam = pd.read_csv('SMSSpamCollection', sep = '\t', header = None, names = ['Label', 'SMS'])

In [2]:
sms_spam.shape

(5572, 2)

In [3]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam_dist = sms_spam['Label'].value_counts(normalize = True) * 100
spam_dist

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The dataset has 5,572 rows of SMS and labels for spam or ham. Almost 87% of the dataset is non-spam(ham) and a little over 13% is spam.

## Training & Test Set



Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

In [5]:
samples = sms_spam.sample(frac = 1, random_state = 1)

In [6]:
sms_train = samples.iloc[:4458].reset_index(drop = True)
sms_test = samples.iloc[4458:].reset_index(drop = True)

In [7]:
train_dist = sms_train['Label'].value_counts(normalize = True)*100
train_dist

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [8]:
test_dist = sms_test['Label'].value_counts(normalize = True)* 100
test_dist

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Both the train set and the test set have the 87/13 split between non-spam and spam. This is similar to the original dataset.

## Letter Case & Punctuation

To make the calculations easier, we will clean the dataset and transform the format.

In [9]:
sms_train.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
sms_train['SMS'] = sms_train['SMS'].copy().str.replace('\W',' ')
sms_train['SMS'] = sms_train['SMS'].str.lower()
sms_train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Creating the Vocabulary
Let's create a list with all of the unique words that occur in the messages of our training set.

In [11]:
sms_train['SMS'] = sms_train['SMS'].str.split()
sms_train.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [12]:
vocabulary = []
for msg in sms_train['SMS']:
    for word in msg:
        vocabulary.append(word)
        

In [13]:
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

In [14]:
word_counts_per_sms = {unique_word: [0] * len(sms_train['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(sms_train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [15]:
words_df = pd.DataFrame(word_counts_per_sms)
words_df.head()

,mnth,housewives,want,swoop,thinks,comuk,1450,raji,his,physics,...,fighting,checkboxes,baig,influx,group,timi,frying,cheese,seemed,box434sk38wp150ppm18
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
sms_train.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [17]:
sms_train_clean = pd.concat([sms_train, words_df], axis = 1)
sms_train_clean.head()

,Label,SMS,mnth,housewives,want,swoop,thinks,comuk,1450,raji,...,fighting,checkboxes,baig,influx,group,timi,frying,cheese,seemed,box434sk38wp150ppm18
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First


In [36]:
spam_df = sms_train_clean[sms_train_clean['Label'] == 'spam']
ham_df = sms_train_clean[sms_train_clean['Label'] == 'ham']

p_spam = len(spam_df) / len(sms_train_clean)
p_ham = len(ham_df) / len(sms_train_clean)

In [37]:
n_spam = spam_df.apply(len).sum()
n_ham = ham_df.apply(len).sum()

In [38]:
n_vocab = len(vocabulary)
alpha = 1

## Calculating Parameters

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

- P("lost"|Spam) and P("lost"|Ham)
- P("navigate"|Spam) and P("navigate"|Ham)
- P("sea"|Spam) and P("sea"|Ham)
We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm will be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

In [39]:
parameters_spam = {unique_word : 0 for unique_word in vocabulary}
parameters_ham = {unique_word : 0 for unique_word in vocabulary}

In [40]:
for word in vocabulary:
    n_w_spam = spam_df[word].sum()
    p_spam_word = (n_w_spam + alpha) / (n_spam + alpha*n_vocab)
    parameters_spam[word] = p_spam_word
    
    n_w_ham = ham_df[word].sum()
    p_ham_word = (n_w_ham + alpha) / (n_spam + alpha*n_vocab)
    parameters_ham[word] = p_ham_word

## Classifying a New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn)
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [41]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [42]:
test_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_2 = "Sounds good, Tom, then see u there"

In [43]:
test_1_classified = classify(test_1)
test_1_classified

P(Spam|message): 2.2361181244033363e-46
P(Ham|message): 3.7437982150291947e-44
Label: Ham


In [44]:
test_2_classified = classify(test_2)
test_2_classified

P(Spam|message): 1.676839627472796e-41
P(Ham|message): 3.690915025598831e-34
Label: Ham


## Measuring the Spam Filter's Accuracy

On the previous screen, we managed to create a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

In [45]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [46]:
sms_test['predictions'] = sms_test['SMS'].apply(classify_test_set)
sms_test.head()

,Label,SMS,predictions
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [47]:
correct = 0
total = len(sms_test)
for index, row in sms_test.iterrows():
    if row['Label'] == row['predictions']:
        correct += 1
        
accuracy = correct / total
accuracy

0.9443447037701975

The accuracy of the spam filter is 94% which is good for a test set of over 1,000 messages. Our initial goal was to attain an accuracy of over 80% and we have successfully accomplished it.

Next steps include:

- Analyzing the incorrect predictions and add to the algorithm to increase accuracy. 